In [1]:
# importar bibliotecas
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score

import scipy.stats

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from IPython.display import display,HTML
import seaborn as sns


### Detalhes do Dataset utilizado


In [2]:
#configurar tema dos gráficos
sns.set_theme()
# abrir dataset com os indicadores
MIN_CLUSTERS=2
MAX_CLUSTERS=15

PATH_TO_DATASET="../dataset/dados_k_means_3.csv"
dfArquivo=pd.read_csv(PATH_TO_DATASET,sep=";")
dfArquivo = dfArquivo.drop(["EN1.1"],axis=1)

# Excluir coluna do identificador censitário
dados=dfArquivo.drop(["Census tract"],axis=1)
dfArquivo.describe()

FileNotFoundError: [Errno 2] No such file or directory: '../dataset/dados_k_means_3.csv'

### Utilização do GAP Statistic para determinar o número ótimo de clusters

O Gap Statistic é uma medida estatística usada para determinar o número ideal de clusters em uma análise de agrupamento (clustering). Ele compara a dispersão dentro dos clusters com a dispersão esperada em um conjunto de dados aleatório com o mesmo número de clusters. Uma lacuna maior indica uma boa separação entre os clusters e sugere que o número correspondente de clusters é uma escolha apropriada.

A interpretação do Gap Statistic no contexto de encontrar o número ideal de clusters é a seguin:



Gap Estatisticamente Significa:

vo: Se o Gap Statistic para um determinado número de clusters for significativamente maior do que para o número de clusters adjacentes, isso sugere que o número de clusters é apropriado para a estrutura subjacente dos dad
s.

Gap Estatisticamente não Significa

tivo: Se o Gap Statistic não mostrar um aumento significativo para um determinado número de clusters em comparação com o próximo, pode indicar que os dados não contêm estrutura de cluster substancial.

Em resumo, ao analisar o gráfico Gap Statistic vs. K, você procura um ponto onde a lacuna entre os valores da estatística Gap seja maximizada, indicando que adicionar mais clusters não melhora substancialmente a qualidade da clusterização. Isso ajuda a determinar o número ótimo de clusters para o seu conjunto de dados.e dados.

In [ ]:
# Determinar a quantidade ótima de clusters usando GAP Statitisc

def gap_statistic_method(data, nrefs=3,min_clusters=2, max_clusters=15):
    """
    Calcular a quantidade ótima de clusters utilizando o método Gap Statistic 
    Params:
        data: Conjunto de dados (DataFrame ou ndarray)
        nrefs: Quantidade de conjuntos de referência que serão criados
        maxClusters: Número máximo de clusters que serão testados
    Return: (melhor_k, resultsDf)
    """
    
    if isinstance(data, pd.DataFrame):
        data = data.values 
        
    gaps = np.zeros((len(range(min_clusters, max_clusters)),))
    resultsDf = pd.DataFrame({'qtd_clusters':[], 'gap':[]})
    
    for gap_index, k in enumerate(range(min_clusters, max_clusters)):
        # Acumulador dos resultados de dispersão
        refDisps = np.zeros(nrefs)
        
        # Para n referências, gere uma amostra aleatória e execute o algoritmo K-means, 
        # obtendo a dispersão resultante de cada iteração.
        for i in range(nrefs):
            # Criar conjunto aleatório de referência
            randomReference = np.random.random_sample(size=data.shape)
            
            # Aplicar o K-means no conjunto de referência
            km = KMeans(k)
            km.fit(randomReference)
            refDisp = km.inertia_
            refDisps[i] = refDisp
            
        # Aplicar o K-means no conjunto de dados original e calculaar a dispersão
        km = KMeans(k)
        km.fit(data)
        origDisp = km.inertia_
        
        # Calcular o GAP Statistic
        gap = np.log(np.mean(refDisps)) - np.log(origDisp)
        
        # Armazenar o resultado do GAP Statistic
        gaps[gap_index] = gap
        resultsDf.loc[gap_index] = [k, gap] 
        
    indice_melhor_k = gaps.argmax()
    return resultsDf.iloc[indice_melhor_k]["qtd_clusters"],indice_melhor_k, resultsDf


In [ ]:
# Definir o conjunto de dados para avaliação do GAP Statistic
# Excluir coluna do identificador censitário
dados=dfArquivo.drop(["Census tract"],axis=1)

# Executar o GAP Statistic para determinar o número ótimo de clusters
melhor_k,indice_maior_gap, resultsDf = gap_statistic_method(dados.values,
                                                          min_clusters=MIN_CLUSTERS,
                                                          max_clusters=MAX_CLUSTERS)

# Plotagem dos resultados
sns.lineplot(x=resultsDf["qtd_clusters"], y=resultsDf["gap"], marker='o')

plt.xlabel('Número de clusters')
plt.ylabel('Gap Statistic')
plt.title('Avaliação de Clusters com Gap Statistic')
plt.axvline(x=melhor_k, color='red', linestyle='--', label=f'Número ideal de clusters: {melhor_k}')
plt.legend()
plt.show()

# Imprime o número ideal de clusters e os valores do Gap Statistic
print("Número ideal de clusters:", melhor_k)
print("Valores do Gap Statistic:\n\n", resultsDf)

### Utilização do Método de Cotovelo para determinar o número ótimo de clusters

O método do cotovelo (Elbow Method) é uma técnica simples e comum para determinar o número ideal de clusters em um conjunto de dados. 
Consiste em executar o algoritmo de clustering (por exemplo, K-means) para diferentes valores de K e plotar a soma dos quadrados das distâncias intra-cluster em relação ao número de clusters. 

O ponto de "cotovelo" no gráfico é geralmente considerado como o número ideal de clusters.

A lógica por trás do Método do Cotovelo é que, à medida que o número de clusters aumenta, a soma dos quadrados das distâncias intra-cluster diminui. No entanto, em um certo ponto, adicionar mais clusters não leva a uma redução significativa na soma dos quadrados das distâncias intra-cluster. Este ponto é comumente referido como o ponto de "cotovelo".


In [ ]:
# Determinar a quantidade ótima de clusters usando o Método de Cotovelo (Elbow Method)
def elbow_method(data, min_clusters=2, max_clusters=10):
    """
    Implementação do método de cotovelo para encontrar o número ideal de clusters.
    
    Params:
        data: ndarray de forma (n_samples, n_features)
            O conjunto de dados a ser agrupado.
        max_clusters: int, opcional (padrão=10)
            O número máximo de clusters a serem testados.
    
    Return:
        (indice_maior_elbow, resultsDf)
    """
    distortions = []  # Armazena a soma dos quadrados das distâncias intra-cluster
    resultsDf = pd.DataFrame({'qtd_clusters':[], 'distortions':[]})
    
    for i,k in enumerate(range(min_clusters, max_clusters + 1)):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(data)
        
        # Armazenar o resultado do soma dos quadrados das distâncias intra-cluster
        distortions.append(kmeans.inertia_) 
        resultsDf.loc[i] = [k, kmeans.inertia_] 
    
    # Encontrar o ponto de cotovelo (método mais simples - procurando a maior inclinação)
    differences = np.diff(distortions)
    indice_melhor_k = np.argmax(differences)
    
    return resultsDf.iloc[indice_melhor_k]["qtd_clusters"],indice_melhor_k, resultsDf, differences


In [ ]:
### Definir o conjunto de dados para avaliação do método de cotovelo
# Excluir coluna do identificador censitário
dados=dfArquivo.drop(["Census tract"],axis=1)

# Executar método de cotovelo para determinar o número ótimo de clusters
melhor_k,elbow_index, resultsDf,differences = elbow_method(dados,
                                                          min_clusters=MIN_CLUSTERS,
                                                          max_clusters=MAX_CLUSTERS)

sns.lineplot(x=resultsDf["qtd_clusters"], y=resultsDf["distortions"], marker='o')

plt.xlabel('Número de clusters')
plt.ylabel('Soma dos quadrados das distâncias intra-cluster')
plt.title('Método do Cotovelo')
plt.axvline(x=melhor_k, color='red', linestyle='--', label=f'Número ideal de clusters: {melhor_k}')  # Adiciona uma linha vertical
plt.legend()
plt.show()

# Imprime o número ideal de clusters e os valores do Gap Statistic
print("Número ideal de clusters:", melhor_k)
print("Valores do Elbow Method:\n\n", resultsDf,"\nDiferenças: \n",pd.DataFrame(differences))


### Utilização do Método Silhouette Score para determinar o número ótimo de clusters

O Silhouette Score é uma medida que avalia a qualidade dos clusters formados por um algoritmo de clustering, como o K-means. 
Ele fornece uma pontuação para cada ponto de dados, indicando o quão bem esse ponto é atribuído ao seu cluster e quão distante ele está dos outros clusters. 
O Silhouette Score varia de -1 a 1, onde valores mais altos indicam uma melhor separação entre os clusters.

In [ ]:
def silhouette_score_method(data, min_clusters=2 ,max_clusters=15):
    """
    Implementação do Silhouette Score para determinar o número ideal de clusters usando K-means.
    
    Params:
        data: ndarray de forma (n_samples, n_features)
            O conjunto de dados a ser agrupado.
        min_clusters: int, opcional (padrão=3)
            O número mínimo de clusters a serem testados.
        max_clusters: int, opcional (padrão=10)
            O número máximo de clusters a serem testados.
    
    Return:
        indice_maior_silhueta: int
            O número ideal de clusters com base no Silhouette Score.
        resultsDf: DataFrame
            Dataframe contendo os Silhouette Scores para cada número de clusters testado.
    """
    
    silhouette_scores = []
    resultsDf = pd.DataFrame({'qtd_clusters':[], 'silhouette_score':[]})
    
    for i,k in enumerate(range(min_clusters, max_clusters + 1)):
        kmeans = KMeans(n_clusters=k, random_state=42)
        cluster_labels = kmeans.fit_predict(data)
        silhouette_avg = silhouette_score(data, cluster_labels)
        
        # Armazenar o resultado da silhueta média
        silhouette_scores.append(silhouette_avg)
        resultsDf.loc[i] = [k, silhouette_avg] 
    
    indice_melhor_k = np.argmax(silhouette_scores)  
    return resultsDf.iloc[indice_melhor_k]["qtd_clusters"],indice_melhor_k, resultsDf



In [ ]:
# Definir o conjunto de dados para avaliação do método de silhueta
# Excluir coluna do identificador censitário
dados=dfArquivo.drop(["Census tract"],axis=1)

melhor_k,indice_melhor_k, resultsDf = silhouette_score_method(dados,
                                                            min_clusters=MIN_CLUSTERS,
                                                            max_clusters=MAX_CLUSTERS)

# Plotagem dos resultados
sns.lineplot(x=resultsDf["qtd_clusters"], y=resultsDf["silhouette_score"], marker='o')
plt.xlabel('Número de clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score para K-means')
plt.axvline(x=melhor_k, color='red', linestyle='--', label=f'Número ideal de clusters: {melhor_k}')
plt.legend()
plt.show()

# Imprime o número ideal de clusters e os Silhouette Scores
print("Número ideal de clusters:", melhor_k)
print("Silhouette Scores:\n", resultsDf)

### Utilização do Método Calinski-Harabasz Index para determinar o número ótimo de clusters

O índice de Calinski-Harabasz é uma métrica interna que avalia a qualidade dos clusters formados por um algoritmo de clustering, como o K-means. 
Ele calcula a razão entre a dispersão intra-cluster e a dispersão inter-cluster. 
Quanto maior o valor do índice de Calinski-Harabasz, melhor a separação entre os clusters.

In [ ]:
def calinski_harabasz_index_method(data,min_clusters=3,max_clusters=10):
    """
    Implementação do índice de Calinski-Harabasz para determinar o número ideal de clusters usando K-means.
    
    Params:
        data: ndarray de forma (n_samples, n_features)
            O conjunto de dados a ser agrupado.
        min_clusters: int, opcional (padrão=3)
            O número mínimo de clusters a serem testados.
        max_clusters: int, opcional (padrão=10)
            O número máximo de clusters a serem testados.
    
    Return:
        best_k: int
            O número ideal de clusters com base no índice de Calinski-Harabasz.
        calinski_scores: list
            Lista contendo os índices de Calinski-Harabasz para cada número de clusters testado.
    """
    calinski_scores = []
    resultsDf = pd.DataFrame({'qtd_clusters':[], 'calinski_score':[]})
    
    for i,k in enumerate(range(min_clusters, max_clusters + 1)):
        kmeans = KMeans(n_clusters=k, random_state=42)
        cluster_labels = kmeans.fit_predict(data)
        calinski_score = calinski_harabasz_score(data, cluster_labels)
        
        # Armazenar o resultado dos scores
        calinski_scores.append(calinski_score)
        resultsDf.loc[i] = [k, calinski_score] 
    
    indice_melhor_k = np.argmax(calinski_scores) 
    return resultsDf.iloc[indice_melhor_k]["qtd_clusters"],indice_melhor_k, resultsDf



In [ ]:
# Definir o conjunto de dados para avaliação do método de silhueta
# Excluir coluna do identificador censitário
dados=dfArquivo.drop(["Census tract"],axis=1)

melhor_k,indice_melhor_k, resultsDf = calinski_harabasz_index_method(dados,MIN_CLUSTERS,MAX_CLUSTERS)

# Plotagem dos resultados
sns.lineplot(x=resultsDf["qtd_clusters"], y=resultsDf["calinski_score"], marker='o')

plt.xlabel('Número de clusters')
plt.ylabel('Índice de Calinski-Harabasz')
plt.title('Índice de Calinski-Harabasz para K-means')
plt.axvline(x=melhor_k, color='red', linestyle='--', label=f'Número ideal de clusters: {melhor_k}')
plt.legend()
plt.show()

# Imprime o número ideal de clusters e os índices de Calinski-Harabasz
print("Número ideal de clusters:", melhor_k)
print("Índices de Calinski-Harabasz:\n", resultsDf)

### Utilização do Davies Bauldin Index para determinar o número ótimo de clusters

O Índice Davies-Bouldin é uma métrica interna usada para avaliar a qualidade dos clusters em um algoritmo de clustering. 
Ele é calculado como a média do grau de similaridade entre cada cluster e seu cluster mais próximo, onde um índice menor indica uma melhor separação entre os clusters.

In [ ]:
def davies_bouldin_index_method(data, min_clusters=3,max_clusters=10):
    """
    Implementação do Índice Davies-Bouldin para determinar o número ideal de clusters usando K-means.
    
    Params:
        data: ndarray de forma (n_samples, n_features)
            O conjunto de dados a ser agrupado.
        max_clusters: int, opcional (padrão=10)
            O número máximo de clusters a serem testados.
    
    Return:
        best_k: int
            O número ideal de clusters com base no Índice Davies-Bouldin.
        davies_bouldin_scores: list
            Lista contendo os Índices Davies-Bouldin para cada número de clusters testado.
    """
    davies_bouldin_scores = []
    resultsDf = pd.DataFrame({'qtd_clusters':[], 'davies_bouldin_score':[]})
    for i,k in enumerate(range(min_clusters, max_clusters + 1)):
        kmeans = KMeans(n_clusters=k, random_state=42)
        cluster_labels = kmeans.fit_predict(data)
        davies_bouldin_score_value = davies_bouldin_score(data, cluster_labels)

        # Armazenar o resultado dos scores
        davies_bouldin_scores.append(davies_bouldin_score_value)
        resultsDf.loc[i] = [k, davies_bouldin_score_value] 
    
    indice_melhor_k = np.argmin(davies_bouldin_scores) 
    return resultsDf.iloc[indice_melhor_k]["qtd_clusters"],indice_melhor_k, resultsDf


In [ ]:
# Definir o conjunto de dados para avaliação do método de silhueta
# Excluir coluna do identificador censitário
dados=dfArquivo.drop(["Census tract"],axis=1)

melhor_k,indice_melhor_k, resultsDf = davies_bouldin_index_method(dados,MIN_CLUSTERS,MAX_CLUSTERS)

# Plotagem dos resultados
sns.lineplot(x=resultsDf["qtd_clusters"], y=resultsDf["davies_bouldin_score"], marker='o')

plt.xlabel('Número de clusters')
plt.ylabel('Índice Davies-Bouldin')
plt.title('Índice Davies-Bouldin para K-means')
plt.axvline(x=melhor_k, color='red', linestyle='--', label='Número ideal de clusters: {}'.format(melhor_k))
plt.legend()
plt.show()

# Imprime o número ideal de clusters e os Índices Davies-Bouldin
print("Número ideal de clusters:", melhor_k)
print("Índices Davies-Bouldin: \n", resultsDf["davies_bouldin_score"])

### Comparação dos métodos para definição do número ótimo de clusters

In [ ]:
# Definir o conjunto de dados para avaliação do método de silhueta
# Excluir coluna do identificador censitário
dados=dfArquivo.drop(["Census tract"],axis=1)

resultadosMetodos=pd.DataFrame({'metodo':[] ,'qtd_clusters':[]})

res_gap_stats = gap_statistic_method(dados.values,
                                  min_clusters=MIN_CLUSTERS,
                                  max_clusters=MAX_CLUSTERS)
res_elbow = elbow_method(dados,MIN_CLUSTERS,MAX_CLUSTERS)
res_calinski=calinski_harabasz_index_method(dados,MIN_CLUSTERS,MAX_CLUSTERS)
res_davies= davies_bouldin_index_method(dados,MIN_CLUSTERS,MAX_CLUSTERS)
res_silhouett=silhouette_score_method(dados,MIN_CLUSTERS,MAX_CLUSTERS)

resultadosMetodos.loc[len(resultadosMetodos.index)] = ['Gap Statistc',res_gap_stats[0] ] 
resultadosMetodos.loc[len(resultadosMetodos.index)] = ['Elbow',res_elbow[0] ]  
resultadosMetodos.loc[len(resultadosMetodos.index)] = ['Calinski',res_calinski[0] ]  
resultadosMetodos.loc[len(resultadosMetodos.index)] = ['Davies',res_davies[0] ]  
resultadosMetodos.loc[len(resultadosMetodos.index)] = ['Silhouette',res_silhouett[0] ]  


resultadosMetodos["qtd_clusters"]

In [ ]:
import seaborn as sns

g = sns.catplot(
    data=resultadosMetodos, 
    kind="bar",
    x="metodo", 
    y="qtd_clusters", palette="tab10",hue="metodo"
)

g.set_axis_labels("Método", "Qtd. Clusters")

plt.gca().set_yticks([])
plt.title("Avaliacão métodos definição melhor número de Clusters\n")

for i,k in enumerate(resultadosMetodos["qtd_clusters"]):
    plt.text(i, k + 0.1, str(int(k)), ha='center', va='bottom')

plt.show()

### Avaliação da qualidade dos agrupamentos

In [ ]:
silhouette_scores = []
resultsDf = pd.DataFrame({'qtd_clusters':[], 'silhouette_score':[]})

classificacao=[]
silhuetas=[]

for i,k in enumerate([2,8,13]):
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(dados)
    silhouette_avg = silhouette_score(dados, cluster_labels)
    ss = silhouette_samples(dados, cluster_labels)
    
    # Armazenar o resultado da silhueta média
    silhouette_scores.append(silhouette_avg)
    classificacao.append(cluster_labels)
    silhuetas.append(ss)
    resultsDf.loc[i] = [k, silhouette_avg] 



In [ ]:
# Plotagem dos resultados
sns.barplot(x=resultsDf["qtd_clusters"], y=resultsDf["silhouette_score"])
plt.xlabel('Número de clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score para K-means')
#plt.axvline(x=melhor_k, color='red', linestyle='--', label=f'Número ideal de clusters: {melhor_k}')
plt.legend()
plt.show()

# Imprime o número ideal de clusters e os Silhouette Scores
print("Número ideal de clusters:", melhor_k)
print("Silhouette Scores:\n", resultsDf)

In [ ]:
rotulos_silhuetas = pd.DataFrame({'arranjo':[],'rotulo':[], 'silhouette_score':[]})
for i in range(len(classificacao)):
    for j in range(len(classificacao[i])):
        rotulos_silhuetas.loc[len(rotulos_silhuetas.index)]=[int(i)+1,int(classificacao[i][j]),silhuetas[i][j]]
    
# rotulos_silhuetas.sort_values(["arranjo","rotulo"],inplace=True)

In [ ]:

rotulos_silhuetas[rotulos_silhuetas["arranjo"]!=4] 
    

In [ ]:
sns.set()
g = sns.FacetGrid(rotulos_silhuetas, col="arranjo")
g.map(sns.histplot, "silhouette_score")